In [2]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 461 kB/s 
     |████████████████████████████████| 448 kB 58.8 MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100
from konlpy.tag import Okt
from collections import Counter

import os
from tqdm import tqdm_notebook

In [4]:
def merge_df(df):
    # 데이터 merge
#     df = total_df[0].copy()
    df.fillna(0,inplace=True)
    df.replace(0,'',inplace=True)
    df['merge'] = df['title'] + df['cat1'] + df['cat2'] + df['cat3']
    
    # 작업용 df로 변경
    df = df[['merge','keyword']]
    return df

## 작업파일 불러오기

In [5]:
#내 작업
query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시']

In [7]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/content/drive/MyDrive/test/{query_list[i]}.csv'))

In [8]:
bike = total_df[3]
bike

,product_id,title,keyword,cat1,cat2,cat3,view,image
0,168841496,오토바이 배달가방,"배달가방,가방,배달,오토바이,스쿠터",차량/오토바이,오토바이/스쿠터,오토바이(125cc 이하),375,https://media.bunjang.co.kr/product/168841496_...
1,151562879,부산 오토바이 급처 다 ㅅ,"보이져,조이맥스,pcx,오토바이,스쿠터",차량/오토바이,오토바이/스쿠터,오토바이(125cc 이하),7790,https://media.bunjang.co.kr/product/151562879_...
2,139316718,오토바이 급처다삽니다,"보이져,조이맥스,pcx,어드리,오토바이",차량/오토바이,오토바이/스쿠터,오토바이(125cc 이하),8676,https://media.bunjang.co.kr/product/139316718_...
3,174557776,세나 20s / 세나20s evo 오토바이 용퓸,"세나,세나20 세나20,세나20s evo",차량/오토바이,오토바이 용품/부품,라이더 용품,13,https://media.bunjang.co.kr/product/174557776_...
4,173938900,오토바이 블루투스 헬멧,"오토바이,하이하,헬멧,블루투스",차량/오토바이,오토바이 용품/부품,라이더 용품,217,https://media.bunjang.co.kr/product/173938900_...
...,...,...,...,...,...,...,...,...
5995,152671110,슈퍼커브110 15년식 배달오토바이 상태좋음오늘만 급처,"배달오토바이,딸배,슈퍼커브",차량/오토바이,오토바이/스쿠터,오토바이(125cc 이하),1059,https://media.bunjang.co.kr/product/152671110_...
5996,130007117,일파인스타 오토바이 슈트 유럽50사이즈,"알파인스타,알파인스타슈트,오토바이슈트,슈트,바이크슈트",차량/오토바이,오토바이 용품/부품,라이더 용품,2358,https://media.bunjang.co.kr/product/130007117_...
5997,154062958,125오토바이삽니다,NaN,차량/오토바이,오토바이/스쿠터,오토바이(125cc 이하),83,https://media.bunjang.co.kr/product/154062958_...
5998,150101780,오토바이 라이트,"할리데이비슨,오토바이,라이트,안개등",차량/오토바이,오토바이/스쿠터,오토바이(125cc 초과),199,https://media.bunjang.co.kr/product/150101780_...


## 전처리

In [9]:
bike['cat1'].unique()

array(['차량/오토바이', '스포츠/레저', '디지털/가전', '키덜트', '패션 액세서리', '가구/인테리어', '기타',
       '남성의류', '신발', '시계/쥬얼리', '뷰티/미용', '생활/가공식품', '재능', '지역 서비스',
       '유아동/출산', '예술/희귀/수집품', '여성의류', '도서/티켓/문구', '가방', '커뮤니티', '번개나눔',
       '구인구직'], dtype=object)

In [10]:
# # cat1
# adidas = adidas[(adidas['cat1'] == '스포츠/레저') | (adidas['cat1'] == '신발') | (adidas['cat1'] == '기타')]
# # adidas.index= range(len(adidas))
# adidas

In [11]:
bike.cat2.unique()

array(['오토바이/스쿠터', '오토바이 용품/부품', '전동킥보드/전동휠', '차량 용품/부품', '오디오/영상/관련기기',
       '자전거', '피규어/인형', '목도리/장갑', '인테리어', '겨울 스포츠', '0', '청바지', '스니커즈',
       '쥬얼리', '스킨케어', '레고/블럭', '생활용품', '바지', '가전제품', '국산차', '생활서비스/지식',
       '이사/용달', '유아동용품', '맨투맨', '수입차', '패딩/점퍼', '인라인/스케이트보드', '주방용품',
       '등산/클라이밍', '자켓', '골동품', '카메라/DSLR', '남성화', '모바일', '도서', '모자',
       '기타 액세서리', '미용소품/기기', '문구', '타이어/휠', '교육/완구/인형', '예술작품', '기타(키덜트)',
       'RC/드론', '기타 구기 스포츠', '남성가방', '기타', '희귀/수집품', '여성화', '프라모델', '캠핑',
       '테마/이벤트', '여성가방', '조끼/베스트', '티셔츠', '산업용 차량/장비', '안경/선글라스', '가구',
       '산업용품', '여행용', '시계', '코트', '후드티/후드집업', '골프', '니트/스웨터', '낚시',
       '광고방/상점공지', '남아의류(3-6세)', '차량/튜닝 용품', '스카프/넥타이', '야구',
       '헬스/요가/필라테스', '원피스', '반바지', '바디/헤어케어', '덤(조건부나눔)', 'PC부품/저장장치',
       '수상 스포츠', '트레이닝', '차량/수리', '셔츠', '알바찾아요'], dtype=object)

In [12]:
# # cat1
# adidas = adidas[(adidas['cat2'] == '축구') | (adidas['cat2'] == '남성화') | (adidas['cat2'] == '스니커즈')
#                 | (adidas['cat2'] == '여성화') | (adidas['cat2'] == '등산/클라이밍')| (adidas['cat2'] == '농구')
#                 | (adidas['cat2'] == '골프')]
# adidas.index= range(len(adidas))
# adidas

In [13]:
bike = merge_df(bike)
bike.rename(columns={'merge':'feature', 'keyword' : 'label'}, inplace=True)
bike

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,feature,label
0,오토바이 배달가방차량/오토바이오토바이/스쿠터오토바이(125cc 이하),"배달가방,가방,배달,오토바이,스쿠터"
1,부산 오토바이 급처 다 ㅅ차량/오토바이오토바이/스쿠터오토바이(125cc 이하),"보이져,조이맥스,pcx,오토바이,스쿠터"
2,오토바이 급처다삽니다차량/오토바이오토바이/스쿠터오토바이(125cc 이하),"보이져,조이맥스,pcx,어드리,오토바이"
3,세나 20s / 세나20s evo 오토바이 용퓸차량/오토바이오토바이 용품/부품라이더 용품,"세나,세나20 세나20,세나20s evo"
4,오토바이 블루투스 헬멧차량/오토바이오토바이 용품/부품라이더 용품,"오토바이,하이하,헬멧,블루투스"
...,...,...
5995,슈퍼커브110 15년식 배달오토바이 상태좋음오늘만 급처차량/오토바이오토바이/스쿠터오...,"배달오토바이,딸배,슈퍼커브"
5996,일파인스타 오토바이 슈트 유럽50사이즈차량/오토바이오토바이 용품/부품라이더 용품,"알파인스타,알파인스타슈트,오토바이슈트,슈트,바이크슈트"
5997,125오토바이삽니다차량/오토바이오토바이/스쿠터오토바이(125cc 이하),
5998,오토바이 라이트차량/오토바이오토바이/스쿠터오토바이(125cc 초과),"할리데이비슨,오토바이,라이트,안개등"


### 전처리(라벨만 따로 묶어주기)

In [14]:
# # label 가장 많은 것 확인
# long = 0
# for i in range(len(bike)):
#     if len(bike.label[i].split(',')) > long:
#         long = len(bike.label[i].split(','))
# long

5

In [27]:
# # lable 5개 각각 columns 으로 지정
# for i in range(1,6):
#     bike['label' + str(i)] = 0
# for i in range(len(bike)):
#     try:
#         bike.label1[i] = bike.label[i].split(',')[0]
#     except: continue
#     try:
#         bike.label2[i] = bike.label[i].split(',')[1]
#     except: continue
#     try:
#         bike.label3[i] = bike.label[i].split(',')[2]
#     except: continue
#     try:
#         bike.label4[i] = bike.label[i].split(',')[3]
#     except: continue
#     try:
#         bike.label5[i] = bike.label[i].split(',')[4]
#     except: continue
# bike_label = bike[['label1','label2','label3','label4','label5']]
# bike_label

### 전처리(명사(키워드)만 추출)

In [23]:
info=[]
for i in range(len(bike)):
    info.append(bike['feature'][i])

nouns_tagger = Okt()
nouns=[]
for line in tqdm_notebook(info):
    nouns.append(nouns_tagger.nouns(line))
    
test = []
for i in tqdm_notebook(range(len(nouns))):
    temp=''
    for j in range(len(nouns[i])):
        temp += nouns[i][j] + ' '
    test.append(temp)
info = test
info

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/6000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/6000 [00:00<?, ?it/s]

['오토바이 배달 가방 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '부산 오토바이 급처 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '오토바이 급처 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '세 세 오토바이 용퓸 차량 오토바이 오토바이 용품 부품 라이더 용품 ',
 '오토바이 블루투스 헬멧 차량 오토바이 오토바이 용품 부품 라이더 용품 ',
 '부산 경남 양산 울산 김해 초 고속 오토바이 당일 매입 차량 오토바이 오토바이 스쿠터 오토바이 초과 ',
 '오토바이 핸들 커버 브레이크 레버 커버 차량 오토바이 오토바이 용품 부품 기타 오토바이 용품 부품 ',
 '오토바이 킥 보드 봉지 걸이 제품 스포츠 레저 킥 보드 휠 ',
 '오토바이 바이크 장갑 차량 오토바이 오토바이 용품 부품 라이더 용품 ',
 '오토바이 킥 보드 레 워머 차량 오토바이 오토바이 용품 부품 라이더 용품 ',
 '오토바이 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '맥스 야마하 오토바이 판매 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '전동 스쿠터 스쿠터 전기 오토바이 스포츠 레저 킥 보드 휠 ',
 '스즈키 오토바이 그린 차량 오토바이 차량 용품 부품 차량 튜닝 용품 ',
 '더 블랙 가능 오토바이 헬멧 블루투스 방수 헤드셋 용량 배터리 링크 차량 오토바이 오토바이 용품 부품 라이더 용품 ',
 '부산 혼다 실버 윙 창원 김해 울산 대구대 인천 경기 부산 오토바이 차량 오토바이 오토바이 스쿠터 오토바이 초과 ',
 '부산 이맥스 제트 플러스 양산 김해 전라 대전 울산 부산 오토바이 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '부산 스즈키 대구대 전산 양산 김해 부산 오토바이 차량 오토바이 오토바이 스쿠터 오토바이 초과 ',
 '부산 이맥스 김해 울산 대구대 부산 오토바이 차량 오토바이 오토바이 스쿠터 오토바이 이하 ',
 '부산 엑스 맥스 울산 양산 김해 경기 부산 오토바이 차량 오토바이 오토바이 스쿠터 오토바이 초과 ',
 '부산 베스파

## TF-IDF로 유사도 구하기

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
countvect = vect.fit_transform(info)

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features = 10000)
tfvect = vect.fit(info)

tfidv_df = pd.DataFrame(tfvect.transform(info).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df

,가격,가공,가구,가능,가도,가드,가디건,가로,가리개,가면,가방,가부토,가성,가스,가와사키,가요,가우스,가을,가이드,가입,가전,가전제품,가정,가젯,가죽,가지,가짜,가처분,가티,각각,각도기,각인,각종,간단,간절,간접,간지,간편,갈매기,갈색,갈치,감성,감지,갑옷,강남,강력,강습,강아지,강원,강제,강철,강태공,개별,개봉,개월,개인,개입,갤럭시,거나,거래,거리,거울,거유,거의,거잭,거제,거제도,거착,거치,건강,건담,걸이,검사,검은색,검정,검정색,게르마늄,게이지,게이트,게임,겨울,겨울스포츠,겨울철,격전,견인,견적,결제,결후,겸용,경고,경기,경기대,경기도,경남,경남대,경량,경보,경보기,경북,경산,...,핸드폰,핸들,핸들바,핸디,핼맷,핼멧,햄버거,햇빛,행거,행사,행유,향수,허드슨,허리,허리케인,헌터,헐멧,헤돈,헤드,헤드라이트,헤드셋,헤드폰,헤론,헤어,헬기,헬로,헬로키티,헬리콥터,헬맷,헬메,헬멜,헬멧,헬멯,헬스,헬스톤스,현금,현대,현장,현찰,협의,형리,호남성,호닝기,호랑이,호수,호스,호원동,호크,호환,혼다,혼다오토바이,홀더,홍진,화물,화방,화성,화이트,화장,확인,확장,환경,환영,환자,활동,활용,황동,황사,회복,회색,회전,회중,회중시계,효과,효성,효성스즈끼,효자,후끈,후드,후루룩,후미,후방,후진,후크,후파,휀다,휀다리스킷,휘도,휠체어,휠터,휴대,휴대폰,흑마,희귀,흰색,히스,히터,히팅,힙색,힙색새거,힙스터
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.605208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.

## 특정 라벨의 유사도 구해보기

In [30]:
# 오토바이만 일단 보자

temp = pd.DataFrame()
for i in range(len(bike)):
    if bike['label1'][i] == '오토바이':
        temp = pd.concat([temp, bike.iloc[[i]]])
    elif bike['label2'][i] == '오토바이':
        temp = pd.concat([temp, bike.iloc[[i]]])
    elif bike['label3'][i] == '오토바이':
        temp = pd.concat([temp, bike.iloc[[i]]])
    elif bike['label4'][i] == '오토바이':
        temp = pd.concat([temp, bike.iloc[[i]]])
    elif bike['label5'][i] == '오토바이':
        temp = pd.concat([temp, bike.iloc[[i]]])
temp = temp[['feature','label']]
temp['label'] = '오토바이'
temp

,feature,label
0,오토바이 배달가방차량/오토바이오토바이/스쿠터오토바이(125cc 이하),오토바이
1,부산 오토바이 급처 다 ㅅ차량/오토바이오토바이/스쿠터오토바이(125cc 이하),오토바이
2,오토바이 급처다삽니다차량/오토바이오토바이/스쿠터오토바이(125cc 이하),오토바이
4,오토바이 블루투스 헬멧차량/오토바이오토바이 용품/부품라이더 용품,오토바이
12,"stoter,전동스쿠터,스쿠터,전기오토바이스포츠/레저전동킥보드/전동휠0",오토바이
...,...,...
5974,오토바이 헬멧차량/오토바이오토바이 용품/부품오토바이/튜닝 용품,오토바이
5975,오토바이 헬멧차량/오토바이오토바이 용품/부품오토바이/튜닝 용품,오토바이
5979,오토바이/바이크 판매합니다차량/오토바이오토바이/스쿠터오토바이(125cc 이하),오토바이
5980,bmw 오토바이 c650 gt모델 팝니다차량/오토바이오토바이/스쿠터오토바이(125c...,오토바이


In [39]:
tfidv_df['label'] = '오토바이'
tf_test = tfidv_df[tfidv_df['label']=='오토바이']
tf_test = tfidv_df.groupby('label').mean()*len(tf_test.columns)
tf_test

,가격,가공,가구,가능,가도,가드,가디건,가로,가리개,가면,가방,가부토,가성,가스,가와사키,가요,가우스,가을,가이드,가입,가전,가전제품,가정,가젯,가죽,가지,가짜,가처분,가티,각각,각도기,각인,각종,간단,간절,간접,간지,간편,갈매기,갈색,갈치,감성,감지,갑옷,강남,강력,강습,강아지,강원,강제,강철,강태공,개별,개봉,개월,개인,개입,갤럭시,거나,거래,거리,거울,거유,거의,거잭,거제,거제도,거착,거치,건강,건담,걸이,검사,검은색,검정,검정색,게르마늄,게이지,게이트,게임,겨울,겨울스포츠,겨울철,격전,견인,견적,결제,결후,겸용,경고,경기,경기대,경기도,경남,경남대,경량,경보,경보기,경북,경산,...,핸드폰,핸들,핸들바,핸디,핼맷,핼멧,햄버거,햇빛,행거,행사,행유,향수,허드슨,허리,허리케인,헌터,헐멧,헤돈,헤드,헤드라이트,헤드셋,헤드폰,헤론,헤어,헬기,헬로,헬로키티,헬리콥터,헬맷,헬메,헬멜,헬멧,헬멯,헬스,헬스톤스,현금,현대,현장,현찰,협의,형리,호남성,호닝기,호랑이,호수,호스,호원동,호크,호환,혼다,혼다오토바이,홀더,홍진,화물,화방,화성,화이트,화장,확인,확장,환경,환영,환자,활동,활용,황동,황사,회복,회색,회전,회중,회중시계,효과,효성,효성스즈끼,효자,후끈,후드,후루룩,후미,후방,후진,후크,후파,휀다,휀다리스킷,휘도,휠체어,휠터,휴대,휴대폰,흑마,희귀,흰색,히스,히터,히팅,힙색,힙색새거,힙스터
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
오토바이,4.765914,3.071844,8.699496,8.796507,0.218383,5.732567,0.138765,0.923155,0.131883,0.125108,20.964092,0.353848,1.442682,0.644614,1.804613,0.153948,0.182107,0.490652,0.287305,0.184811,4.581776,2.080302,0.270174,2.526946,13.280701,0.589044,0.131836,1.064405,1.647665,0.168047,0.607024,0.320257,1.168859,0.144654,0.239261,0.303276,0.355321,0.244241,0.267763,0.170687,0.272406,0.523474,0.182194,0.170035,0.192819,0.314569,0.361453,0.165648,0.548961,0.164624,0.288042,0.207076,0.153505,0.150281,0.204161,5.49655,0.224786,0.723391,0.273469,0.97888,0.608764,0.407514,0.179948,1.502031,1.997712,3.575907,0.410004,0.204911,32.558843,0.541905,0.54638,3.7269,0.740213,0.137666,1.812851,0.772816,0.129612,0.464909,0.219072,0.289086,12.297568,0.130498,0.407038,0.163539,6.704532,1.066855,0.483881,0.189315,1.083997,0.18466,4.874015,0.223046,5.758905,9.012954,0.420556,0.818946,0.210531,1.967926,2.816002,1.089544,...,7.130044,9.586181,1.910101,0.290675,1.743646,2.711301,0.43642,0.131883,0.312741,0.179779,0.884501,0.797186,0.141598,1.038281,0.270067,1.283312,0.182412,0.194789,0.507522,0.46924,0.48007,0.58582,0.182107,0.235514,0.363221,0.204632,0.504191,0.147285,8.337757,0.369307,0.311045,73.273061,0.350541,0.376566,0.32441,8.390485,0.406178,1.154038,0.170812,0.711129,0.200275,0.176072,0.272998,0.302592,0.256804,0.517075,0.163616,0.739384,1.008691,26.135959,2.551562,0.372033,5.396793,11.12413,0.179688,0.197909,2.59593,0.289455,0.155755,0.380913,0.290426,2.161459,0.152631,0.169766,0.19758,0.183334,0.176683,0.362346,0.137666,1.515036,0.109264,0.105781,0.175291,3.429113,0.315943,0.338923,0.203282,1.516442,0.985027,0.775786,0.266884,0.443556,0.358186,0.253327,0.154306,0.286857,0.174119,0.320408,0.235746,0.089349,3.627722,0.271947,8.624506,1.154536,0.142517,0.181245,0.182638,1.973254,0.3046,0.487797
